# 1. Set Up

In [61]:
import pandas as pd
import os
import math

# 2. Creating Columns

In [62]:
import os
from dotenv import load_dotenv
# Load environment variables from the .env file
load_dotenv()

input_path_years = os.getenv("OUTPUT_BILD_PLUS_TON_LISTS_DIR")
output_excel_file = os.getenv("OUTPUT_BILD_PLUS_TON_DIR") + '/SB_AD LIST____filtered 2013 - 2022' + '.xlsx'
dataframe_output = pd.read_excel(output_excel_file)

# 0. classes, 1. ages, 2. genders, 3. emotions, 4. ethnicities, 5. super_categories
unique_predictions_list = [[], [], [], [], [], []]

In [63]:
# Insert unique classes, super_categories, ages, genders, emotions and ethnicities from the first summary in the list
for year in os.listdir(input_path_years):
  year_path = os.path.join(input_path_years, year)
  for ad in os.listdir(year_path):
    if ad.endswith('.xlsx'):
      ad_path = os.path.join(year_path, ad)

      try:
        df = pd.read_excel(ad_path, 'Summary_Objects')
        idx = -1

        for attribute in ['class_name', 'age_group_prediction', 'gender_prediction', 'emotion_prediction', 'ethnicity_prediction', 'super-category']:
          idx += 1
          prediction = list(df[f'{attribute}'])

          for element in prediction:
            if element not in unique_predictions_list[idx]:
              unique_predictions_list[idx].append(element)

      except:
        print(f'AD {ad} has no Summary')

# Remove nan values
for idx in range(6):
  unique_predictions_list[idx] = [x for x in unique_predictions_list[idx] if x == x]

In [64]:
# Insert the column names for the classes and their attributes
for attribute_type in range(5):
  for attribute in unique_predictions_list[attribute_type]:
    dataframe_output[f'{attribute}_count'] = ''
    dataframe_output[f'{attribute}_avg_object_propotion'] = ''
    dataframe_output[f'{attribute}_avg_quadrant_number'] = ''
    dataframe_output[f'{attribute}_frame_ratio'] = ''

In [65]:
# Insert the column names for the human attributes
for super_category in unique_predictions_list[5]:
  dataframe_output[f'super_category_{super_category}_count'] = ''

In [66]:
# Insert the column names for gender-specific speaking time
for category in ['Total', 'Male', 'Female']:
  dataframe_output[f'{category}_Speaking_Time_Seconds'] = ''
  dataframe_output[f'{category}_Speaking_Time_Percent'] = ''

In [67]:
# Insert the column name for the Emotion from Dialogue
dataframe_output['Emotion_from_Dialogue'] = ''

In [68]:
# Insert the column name for the Acoustic Indices
dataframe_output['Acoustic_Complexity_Index__main_value'] = ''
dataframe_output['RMS_energy__mean'] = ''
dataframe_output['compression_ratio_per_second'] = ''

In [69]:
# OUTPUT_BILD_PLUS_TON_DIR
# output to excel
dataframe_output.to_excel(f"{os.getenv('OUTPUT_BILD_PLUS_TON_DIR')}/End_Datei_columns.xlsx", index=False)

# 2. Inserting the data in the table


In [70]:
# Read the input file
input_folder = os.getenv("OUTPUT_BILD_PLUS_TON_LISTS_DIR")

# Read the target file
target_file = f"{os.getenv('OUTPUT_BILD_PLUS_TON_DIR')}/End_Datei_columns.xlsx"
target_df = pd.read_excel(target_file)

## 2.1. Image Analysis

In [71]:
for year in os.listdir(input_folder):
  year_path = os.path.join(input_folder, year)

  for ad in os.listdir(year_path):
    if os.path.isdir(os.path.join(year_path, ad)):
      print(f"Skipping directory: {ad}")
      continue
    
    # Only process Excel files
    if not ad.endswith('.xlsx'):
        continue
    # Get the input file name
    ad_path = os.path.join(year_path, ad)
    print(ad_path)
    input_file_name = os.path.basename(ad_path).split('.')[0]
    # Removing the Title for AD0731
    if input_file_name == 'AD0731_FROMadsoftheworld':
      input_file_name = 'AD0731'

    try:
      input_df = pd.read_excel(ad_path, 'Summary_Objects')

      # Get the first column of the target file
      target_column = target_df.columns[0]
      ads_in_target_column = list(target_df[target_column])

      # Find the matching row in the target file
      if input_file_name in ads_in_target_column:
        target_row = ads_in_target_column.index(input_file_name)
        print(f'Ad {input_file_name} found in the target file!')
      else:
        print(f'Ad {input_file_name} not found in the target file!')

      # Define the attribute groups
      attribute_groups = ['class_name', 'age_group_prediction', 'gender_prediction', 'emotion_prediction', 'ethnicity_prediction', 'super-category']

      # Iterate over the attribute groups in the input file
      for attribute_group in attribute_groups:
          # Check if the attribute group is in the input file
          if attribute_group == 'class_name':
            # Get the class_name column from the input file
            class_name_column = input_df['class_name']
            for index, name in class_name_column.items():
                # Get the class name from the input file
                class_name = name
                for attribute in ['count', 'avg_object_propotion', 'avg_quadrant_number', 'frame_ratio']:
                    # Get the attribute values from the input file
                    attribute_value = input_df[f'{attribute}'][index]

                    # Get the corresponding column name in the target file
                    target_column_name = f'{class_name}_{attribute}'

                    # Write the attribute value to the target file
                    target_df.loc[target_row, target_column_name] = attribute_value

          elif attribute_group == 'super-category':
            # Get the super-category column from the input file
            super_category_column = input_df['super-category']
            for index, name in super_category_column.items():
                # Get the super-category from the input file
                super_category = name
                for attribute in ['count']:
                    # Get the attribute values from the input file
                    attribute_value = input_df[f'{attribute}_{attribute_group}'][index]

                    # Get the corresponding column name in the target file
                    target_column_name = f'super_category_{super_category}_count'

                    # Write the attribute value to the target file
                    target_df.loc[target_row, target_column_name] = attribute_value

          else:
            # Get the attribute-group column from the input file
            attribute_column = input_df[f'{attribute_group}']
            for index, name in attribute_column.items():
                # Get the attribute name from the input file
                attribute_name = name
                for attribute in ['count', 'avg_object_propotion', 'avg_quadrant_number', 'frame_ratio']:
                    # Get the attribute values from the input file
                    attribute_value = input_df[f'{attribute}_{attribute_group}'][index]

                    # Get the corresponding column name in the target file
                    target_column_name = f'{attribute_name}_{attribute}'

                    # Write the attribute value to the target file
                    target_df.loc[target_row, target_column_name] = attribute_value
    except Exception as e:
      print(f'Image Analysis for {input_file_name} was not found!')
      print(e)

/home/arkastor/Development/Commercial-Brand-Differentiating-Message-Analysis/Final_Files/03. Output Bild + Ton/01. output_lists/ADs_IG_2013/AD0253.xlsx
Ad AD0253 found in the target file!
Skipping directory: AD0253
Skipping directory: AD0301
/home/arkastor/Development/Commercial-Brand-Differentiating-Message-Analysis/Final_Files/03. Output Bild + Ton/01. output_lists/ADs_IG_2014/AD0301.xlsx
Ad AD0301 found in the target file!


## 2.2. Audio Analysis

### 2.2.1. Gender speaking time

In [72]:
for year in os.listdir(input_folder):
  year_path = os.path.join(input_folder, year)

  for ad in os.listdir(year_path):
    # Get the input file name
    ad_path = os.path.join(year_path, ad)
    input_file_name = os.path.basename(ad_path).split('.')[0]
    # Removing the Title for AD0731
    if input_file_name == 'AD0731_FROMadsoftheworld':
      input_file_name = 'AD0731'

    try:
      input_df = pd.read_excel(ad_path, 'Gender_speaking_time')

      # Get the first column of the target file
      target_column = target_df.columns[0]
      ads_in_target_column = list(target_df[target_column])

      # Find the matching row in the target file
      if input_file_name in ads_in_target_column:
        target_row = ads_in_target_column.index(input_file_name)
      else:
        print(f'Ad {input_file_name} not found in the target file!')

      metric_column = input_df['Metric']
      for index, name in metric_column.items():
        # Get the metric name from the input file
        metric = str(name)
        for attribute in ['Value_Seconds', 'Value_Percent']:
            # Get the attribute values from the input file
            attribute_value = input_df[f'{attribute}'][index]

            # Target column name
            att = attribute.split('_')[1]
            mtr = metric.split(' ')[0]
            # Get the corresponding column name in the target file
            target_column_name = f'{mtr}_Speaking_Time_{att}'

            # Write the attribute value to the target file
            target_df.loc[target_row, target_column_name] = attribute_value

    except:
      print(f'Audio Analysis for {input_file_name} was not found!')

Audio Analysis for AD0253 was not found!
Audio Analysis for AD0301 was not found!


### 2.2.2. Mood

In [73]:
for year in os.listdir(input_folder):
  year_path = os.path.join(input_folder, year)

  for ad in os.listdir(year_path):
    # Get the input file name
    ad_path = os.path.join(year_path, ad)
    input_file_name = os.path.basename(ad_path).split('.')[0]
    # Removing the Title for AD0731 and merging AD0471_01 into one
    if input_file_name == 'AD0731_FROMadsoftheworld':
      input_file_name = 'AD0731'

    try:
      input_df = pd.read_excel(ad_path, 'Transcription_and_Mood')

      # Get the first column of the target file
      target_column = target_df.columns[0]
      ads_in_target_column = list(target_df[target_column])

      # Find the matching row in the target file
      if input_file_name in ads_in_target_column:
        target_row = ads_in_target_column.index(input_file_name)
      else:
        print(f'Ad {input_file_name} not found in the target file!')

      # Define the attribute
      attribute = 'Emotion'

      # Get the attribute values from the input file
      attribute_value = input_df[f'{attribute}'][0]

      # Get the corresponding column name in the target file
      target_column_name = 'Emotion_from_Dialogue'

      # Write the attribute value to the target file
      target_df.loc[target_row, target_column_name] = attribute_value

    except:
      print(f'Audio Analysis for {input_file_name} was not found!')

Audio Analysis for AD0253 was not found!
Audio Analysis for AD0301 was not found!


### 2.2.3. Acoustic Indicies

In [74]:
for year in os.listdir(input_folder):
  year_path = os.path.join(input_folder, year)

  for ad in os.listdir(year_path):
    # Get the input file name
    ad_path = os.path.join(year_path, ad)
    input_file_name = os.path.basename(ad_path).split('.')[0]

    try:
      input_df = pd.read_excel(ad_path, 'Acoustic_Indices')

      # Get the first column of the target file
      target_column = target_df.columns[0]
      ads_in_target_column = list(target_df[target_column])

      # Find the matching row in the target file
      if input_file_name in ads_in_target_column:
        target_row = ads_in_target_column.index(input_file_name)
      else:
        print(f'Ad {input_file_name} not found in the target file!')

      # Define Metric names
      metric_names = ['Acoustic_Complexity_Index__main_value', 'RMS_energy__mean', 'compression_ratio_per_second']

      # Iterate over the metric names in the input file
      for metric in metric_names:
        # Get the metric values from the input file
        metric_value = input_df[f'{metric}'][0]

        # Get the corresponding column name in the target file
        target_column_name = metric

        # Write the metric value to the target file
        target_df.loc[target_row, target_column_name] = metric_value

    except:
      print(f'Audio Analysis for {input_file_name} was not found!')

Audio Analysis for AD0253 was not found!
Audio Analysis for AD0301 was not found!


## 2.3. Save to Excel

In [75]:
# Remove columns containing "nan_"
target_df = target_df.drop(columns=target_df.columns[target_df.columns.str.contains('nan_')])

# Save the updated target file
# f"{os.getenv('OUTPUT_BILD_PLUS_TON_DIR')}/End_Datei_columns.xlsx"
print(f"Outputting to {os.getenv('OUTPUT_BILD_PLUS_TON_DIR')}/End_Datei.xlsx")
target_df.to_excel(f"{os.getenv('OUTPUT_BILD_PLUS_TON_DIR')}/End_Datei.xlsx", index=False)

Outputting to /home/arkastor/Development/Commercial-Brand-Differentiating-Message-Analysis/Final_Files/03. Output Bild + Ton/End_Datei.xlsx
